In [222]:
import os
import glob
import numpy as np
import pickle
from PIL import Image
from music21 import converter, corpus, instrument, midi, note, chord, pitch

file_name = "edm"

ModuleNotFoundError: No module named 'PIL'

In [217]:
def build_dictionary(file_name):
    notes = []
    chords = []
    offsets = []
    for file in glob.glob(file_name + "/original_data/*.mid"):
        midi = converter.parse(file)
        print("Read in possible notes/chords of %s" % file)
        
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(element.nameWithOctave)
            elif isinstance(element, chord.Chord):
                chords.append(element.commonName)
            else:
                continue
            offsets.append(element.duration.quarterLength)
            
    l = set.union(set(notes), set(chords))   
    mapping_notes = dict([(y,x+1) for x,y in enumerate(sorted(l))])
    mapping_offsets = dict([(y,x+1) for x,y in enumerate(sorted(set(offsets)))])
    
    return mapping_notes, mapping_offsets

In [220]:
def get_midi(music_folder):
    mapping_notes, mapping_offsets = build_dictionary(music_folder)
    
    # save dictionary
    f = open(music_folder + "/element_key_dict.pkl","wb")
    pickle.dump(mapping_notes, f)
    f.close()
    f = open(music_folder + "/offset_key_dict.pkl","wb")
    pickle.dump(mapping_offsets, f)
    f.close()
    
    print("Read in completed. \n")
    
    for file in glob.glob(music_folder + "/original_data/*.mid"):
        midi = converter.parse(file)
        print("Parsing %s" % file)

        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.notes
            
        midi_encoding = []
        max_offset = 0
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                element_key = element.nameWithOctave
            elif isinstance(element, chord.Chord):
                element_key = element.commonName
            else:
                continue
            element_offset = element.offset
            if element_offset*4 > max_offset:
                max_offset = int(element_offset*4)
            element_duration = element.duration
            midi_encoding.append((mapping_notes[element_key], 
                                  int(element_offset*4), 
                                  mapping_offsets[element_duration.quarterLength]))

        file_encoding = np.zeros((1, len(mapping_notes), int(max_offset)))
        print(file_encoding.shape)
        for encoding in midi_encoding:
            file_encoding[0][encoding[0]-1][encoding[1]-1] = encoding[2]/len(mapping_offsets)
        
        # save encoding
        #scipy.misc.imsave(music_folder + "/encodings/" + os.path.basename(file)[:-4] + "_encoding.png", np.array(file_encoding))
        scipy.misc.toimage(
            np.array(file_encoding), cmin=0.0, cmax=1.0
        ).save(music_folder + "/encodings/" + os.path.basename(file)[:-4] + "_encoding.png")

In [221]:
midi_file = get_midi('edm')

Read in possible notes/chords of edm/original_data\deadmau5 - Deus Ex Machina  (midi by Carlo Prato) (www.cprato.com).mid
Read in completed. 

Parsing edm/original_data\deadmau5 - Deus Ex Machina  (midi by Carlo Prato) (www.cprato.com).mid
(1, 8, 575)


AttributeError: module 'scipy.misc' has no attribute 'toimage'

In [62]:
midi_file